In [189]:
# Load packages
import csv
import math
import numpy as np
import os
import pandas as pd
import random

from datetime import datetime as dt
from PIL import Image

## Variables and Hyperparameters

In [219]:
###############
## VARIABLES ##
###############

# name of directory with fire tif files
tif_directory = "toydata"

# name of directory with weather data
weather_directory = 'weather_data'

#####################
## HYPERPARAMETERS ##
#####################

# scale the weather data - yea or nay
scaled_weather = True

# the desired height and width (in pixels) of the matrix to feed into the CNN
# 1 pixel side = 500 meters = 0.310686 miles
matrix_dim = 4

# multiplier for amount of zero-labeled data we want to add to dataset
labeled_multiplier = 2

# test size for train/test split
test_size = 0.2

# training epochs
epoc = 10

## Fire Data Preprocessing

In [220]:
def create_one_hot_matrices(data_dict, fireline):
    '''
    Create matrices for each fire_id that show were the fire was on a given day during the year.
    
    Args:
        - data_dict: a dictionary where the key is "fire_id" and the value is a matrix of pixels 
        triggered by that fire (0, 1)
        - fireline: matrix denoting what day of year that pixel was on fire (1-365)
    Returns:
        - fire_data_dict: a dictionary of the following structure:
            {
                "fire_id": {
                    "day_of_year": one-hot encoded 2D array of fire spread on that day,
                    "day_of_year": one-hot encoded 2D array of fire spread on that day
                }

            }
    '''
    
    fire_data_dict = {}

    for key, val in data_dict.items():
        data = {}
                
        for y in range(1, 366):
            mask = ((fireline == y) & (val == 1))
            mask = mask.astype(int)
        
            if np.sum(mask) > 0:
                data[str(y)] = mask
        
        fire_data_dict[key] = data
        
    return fire_data_dict

In [221]:
def create_day_pairs(fire_data_dict):
    '''
    Create a list of sets where the first value is where the fire was on a given day and the second value is where
    the fire was on the following day.
    
    Args:
        - fire_data_dict: a dictionary of the following structure:
            {
                "fire_id": {
                    "day_of_year": one-hot encoded 2D array of fire spread on that day,
                    "day_of_year": one-hot encoded 2D array of fire spread on that day
                }

            }
    Returns:
        - train_labels: a list of sets where the first value of the set is a one-hot encoded 2D array of fire 
        spread on day_1 and the second value of the set is a one-hot encoded 2D array of fire spread on day_2:
        [
            (one-hot encoded 2D array of fire spread on that day_1, one-hot encoded 2D array of fire spread on day_2),
            (one-hot encoded 2D array of fire spread on that day_2, one-hot encoded 2D array of fire spread on day_3),
        ]
    '''
    
    train_labels = []

    for key, value in fire_data_dict.items():
        burn_matrices = list(value.values())
        day_of_year = list(value.keys())

        for index, day in enumerate(burn_matrices):

            if index < len(burn_matrices) - 1:
                day_1 = burn_matrices[index]
                day_2_index = index + 1
                day_2 = burn_matrices[day_2_index]
                
                doy = day_of_year[day_2_index]
                
                pair = (day_1, day_2)
                train_labels.append((doy, pair))

    return train_labels

In [222]:
def balance_dataset(dataset, matrix_dim, num_pixels, side):
    '''
    Supplement the list produced in `create_labeled_data` with data where there was no data
    
    Args:
        - dataset: a list of sets where the first value of the set is a one-hot encoded 2D array of fire spread 
        on day_1 and the second value of the set is a one-hot encoded 2D array of fire spread on day_2
        - matrix_dim: a hyperparameter for the height and width of the matrices fed into the CNN
        - num_pixels: how many "no-fire" pixel-matrix pairs we want to return
        - side: half the length of the dimension of the outpur matrix
    Returns:
        - no_fire: a list of sets, where the second value (0, 1) represents whether fire is present for a given pixel, and the
        first value is a matrix centered on the second value for the previous day and represents where the fire was
        on the previous day
    '''
        
    no_fire = []

    for (doy, (x, y)) in dataset:    

        x = np.pad(x, pad_width=matrix_dim, mode='constant', constant_values=0)
        y = np.pad(y, pad_width=matrix_dim, mode='constant', constant_values=0)

        vals = np.where(y == 0)
        vals = list(zip(vals[0], vals[1]))

        for (xi, yi) in vals:
            xi_r = xi + side
            xi_l = xi - side
            yi_b = yi + side
            yi_t = yi - side

            m = x[xi_l:xi_r, yi_t:yi_b]
            weather_data = fetch_weather_data(max_values, scaled_weather, doy, xi, yi)
            
            # control for edge cases where shape doesn't match up - not sure why this is happening
            if weather_data is not None and m.shape == (matrix_dim, matrix_dim):
                no_fire.append(((weather_data, m), 0))
    
    no_fire = random.sample(no_fire, num_pixels)
    
    return no_fire

In [223]:
def create_labeled_data(dataset, matrix_dim):
    '''
    Create a list of sets where the first value is a matrix of pixels on a given day and the second value denotes
    whether there was fire in the center pixel on the following day.
    
    Args:
        - dataset: a list of sets where the first value of the set is a one-hot encoded 2D array of fire spread 
        on day_1 and the second value of the set is a one-hot encoded 2D array of fire spread on day_2
        - matrix_dim: a hyperparameter for the height and width of the matrices fed into the CNN
    Returns:
        - data: a list of sets, where the second value (0, 1) represents whether fire is present for a given pixel, and the
        first value is a matrix centered on the second value for the previous day and represents where the fire was
        on the previous day
    '''

    side = int(matrix_dim/2)
    
    data = []
    
    for (doy, (x, y)) in dataset:    

        x = np.pad(x, pad_width=matrix_dim, mode='constant', constant_values=0)
        y = np.pad(y, pad_width=matrix_dim, mode='constant', constant_values=0)

        vals = np.where(y == 1)
        vals = list(zip(vals[0], vals[1]))

        for (xi, yi) in vals:
            xi_r = xi + side
            xi_l = xi - side
            yi_b = yi + side
            yi_t = yi - side

            m = x[xi_l:xi_r, yi_t:yi_b]
            weather_data = fetch_weather_data(max_values, scaled_weather, doy, xi, yi)
            
            if weather_data is not None:
                data.append(((weather_data, m), 1))
    
    data_len = len(data)
    num_pixels = min(int(data_len*labeled_multiplier), data_len)
    
    # balance this dataset with values where there is no fire
    no_fire = balance_dataset(dataset, matrix_dim, num_pixels, side)
    
    # combine and shuffle
    data += no_fire    
    random.shuffle(data)
    
    return data

## Weather Data Preprocessing

In [224]:
def create_weather_dict(directory, scaled_weather):
    '''
    Create a dictionary of weather data from a pickled file
    Args:
        - directory: path to weather pickle file
        - scaled_weather: True/False to scale using max value
    Returns:
        - weather_data: dictionary of key (day of year) and value (dictionary of key (weather parameter) 
        and value (matrix of value for each pixel))
    '''

    path = os.path.abspath(directory)
    
    weather_file = ''
    
    for f in os.listdir(path):
        if f.endswith('.pickle'):
            weather_file = path + '/' + f
    
    weather = pd.read_pickle(weather_file)
    
    weather_dict = {}
    
    for k, v in weather.items():
        weather_dict[k] = {}
        
        for att, matrix in v.items():
            mat = np.nan_to_num(matrix)
            weather_dict[k][att] = mat
     
    weather_data = {}

    for k, v in weather_dict.items():
        doy = dt.strptime(k, "%Y-%m-%d").strftime("%j")
        weather_data[doy] = v
    
    # scale weather data
    vals = list(weather_data.values())[0]
    weather_atts = list(vals.keys())
    max_values = dict.fromkeys(weather_atts, 0)
    
    if scaled_weather == True:
        
        for k, v in weather_dict.items():

            for weather_att, matrix in v.items():
                max_val = matrix.max()
                if max_val > max_values[weather_att]:
                    max_values[weather_att] = max_val
    
    return weather_data, max_values

In [225]:
def fetch_weather_data(max_values, scaled_weather, day_of_year, x, y):
    '''
    Fetch weather data for the relevant day and pixel.
    
    Args:
        - day_of_year: day of the year (1-365)
        - x: x-coordinate of matrix
        - y: y-coordinate of matrix
    Returns:
        - weather_data: an array of relevant weather data for that pixel
    '''
    
    weather_list = []
    
    day_weather = weather_data.get(day_of_year)
    
    if day_weather is None:
        return None
    else:
        for k, v in day_weather.items():
            if scaled_weather == True:
                max_val = max_values.get(k, 1)
                print("X: ",x)
                print("Y: ", y)
                val = v[x,y]/max_val
                print("Val: ", val)
                
                if math.isnan(val):
                    weather_list.append(0)
                else:
                    weather_list.append(val)
            else:
                weather_list.append(v[x,y])
    
    return weather_list

## Prep Dataset for CNN

In [226]:
def prep_dataset_for_cnn(data):
    '''
    Takes a list of (matrix, integer) pairs and returns input data and output labels split into train and test sets
    
    Args:
        - data: a list of (matrix, integer) pairs
    Returns:
        - X: array of input data in matrix_dim X matrix_dim shape
        - Y: array of output labels (0 or 1)
    '''
    
    fire = []
    weather = []
    Y = []

    for ((w, f), y) in data:
        f = np.asarray(f)
        fire.append(f)
        
        w = np.asarray(w)
        weather.append(w)
        
        Y.append(y)

    fire = np.asarray(fire)
    weather = np.asarray(weather)
    Y = np.asarray(Y)
    
    obs = len(fire)
    
    fire = fire.reshape(obs, matrix_dim, matrix_dim, 1)

    return fire, weather, Y

In [227]:
# create a toy dataset to test create_one_matrices

fire_data_dict = {}
fire_data_dict['1'] = np.mat('0 0 0 0 0; 0 1 1 1 0; 0 1 1 0 0; 0 0 1 1 0; 0 1 1 1 1; 0 0 0 1 1')
fireline = np.mat('0 0 0 0 0; 0 154 154 154 0; 0 154 154 0 0; 0 0 155 155 0; 0 155 155 155 155; 0 0 0 156 156')

weather_data, max_values = create_weather_dict(weather_directory, scaled_weather)

fire_data_dict = create_one_hot_matrices(fire_data_dict, fireline)
small_dataset = create_day_pairs(fire_data_dict)
data = create_labeled_data(small_dataset, matrix_dim)
fire, weather, Y = prep_dataset_for_cnn(data)

X:  7
Y:  6
Val:  0.0003337264335293127
X:  7
Y:  6
Val:  nan
X:  7
Y:  6
Val:  0.5684075237068119
X:  7
Y:  6
Val:  0.4939696603466147
X:  7
Y:  6
Val:  0.6782880491214379
X:  7
Y:  6
Val:  0.14109747212695956
X:  7
Y:  6
Val:  0.8128346787142188
X:  7
Y:  6
Val:  0.34626725848292095
X:  7
Y:  7
Val:  0.00032620036296824174
X:  7
Y:  7
Val:  nan
X:  7
Y:  7
Val:  0.5702968031315855
X:  7
Y:  7
Val:  0.49303993513174466
X:  7
Y:  7
Val:  0.6796150706714607
X:  7
Y:  7
Val:  0.1401174153980107
X:  7
Y:  7
Val:  0.8101458892812515
X:  7
Y:  7
Val:  0.3459004055073178
X:  8
Y:  5
Val:  0.0003374881712927132
X:  8
Y:  5
Val:  nan
X:  8
Y:  5
Val:  0.5644634288977811
X:  8
Y:  5
Val:  0.4938612496389944
X:  8
Y:  5
Val:  0.6794398283933635
X:  8
Y:  5
Val:  0.1421271354690892
X:  8
Y:  5
Val:  0.818924162002565
X:  8
Y:  5
Val:  0.3464641516757289
X:  8
Y:  6
Val:  0.0003300451202960734
X:  8
Y:  6
Val:  nan
X:  8
Y:  6
Val:  0.5663202011173256
X:  8
Y:  6
Val:  0.4929413210207456
X:  8
Y: 

/Users/kjohnson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


4
Val:  0.3478693803057086
X:  2
Y:  5
Val:  0.0003600714652022956
X:  2
Y:  5
Val:  nan
X:  2
Y:  5
Val:  0.5767908457992116
X:  2
Y:  5
Val:  0.5000893197687679
X:  2
Y:  5
Val:  0.6645688032969578
X:  2
Y:  5
Val:  0.14182953140867974
X:  2
Y:  5
Val:  0.7985224406457129
X:  2
Y:  5
Val:  0.3474837883375157
X:  2
Y:  6
Val:  0.00035213035642900163
X:  2
Y:  6
Val:  nan
X:  2
Y:  6
Val:  0.5788426379092187
X:  2
Y:  6
Val:  0.4991106186049702
X:  2
Y:  6
Val:  0.666064480458803
X:  2
Y:  6
Val:  0.14077883555010764
X:  2
Y:  6
Val:  0.7957187895659688
X:  2
Y:  6
Val:  0.3470981963693228
X:  2
Y:  7
Val:  0.0003441892476557077
X:  2
Y:  7
Val:  nan
X:  2
Y:  7
Val:  0.5808944300192261
X:  2
Y:  7
Val:  0.4981319174411725
X:  2
Y:  7
Val:  0.6675601576206484
X:  2
Y:  7
Val:  0.13972813969153558
X:  2
Y:  7
Val:  0.7929151384862246
X:  2
Y:  7
Val:  0.3467126044011299
X:  2
Y:  8
Val:  0.00033624813888241364
X:  2
Y:  8
Val:  nan
X:  2
Y:  8
Val:  0.5829462221292333
X:  2
Y:  8
Val:  

Y:  10
Val:  nan
X:  4
Y:  10
Val:  0.5826160588097145
X:  4
Y:  10
Val:  0.49321793421874466
X:  4
Y:  10
Val:  0.6766664358317143
X:  4
Y:  10
Val:  0.1368165120837939
X:  4
Y:  10
Val:  0.7915338146635503
X:  4
Y:  10
Val:  0.34525345744683567
X:  4
Y:  11
Val:  0.0003058937888941412
X:  4
Y:  11
Val:  nan
X:  4
Y:  11
Val:  0.5846028505140461
X:  4
Y:  11
Val:  0.49225882202761134
X:  4
Y:  11
Val:  0.6780946555937131
X:  4
Y:  11
Val:  0.1357940698478573
X:  4
Y:  11
Val:  0.7887761049429459
X:  4
Y:  11
Val:  0.3448753605373734
X:  4
Y:  12
Val:  0.0002981186834831483
X:  4
Y:  12
Val:  nan
X:  4
Y:  12
Val:  0.586589642218378
X:  4
Y:  12
Val:  0.49129970983647786
X:  4
Y:  12
Val:  0.679522875355712
X:  4
Y:  12
Val:  0.13477162761192077
X:  4
Y:  12
Val:  0.7860183952223415
X:  4
Y:  12
Val:  0.3444972636279111
X:  5
Y:  0
Val:  0.00038724111797289233
X:  5
Y:  0
Val:  nan
X:  5
Y:  0
Val:  0.560856134144771
X:  5
Y:  0
Val:  0.5017220933377269
X:  5
Y:  0
Val:  0.665031338683

Y:  2
Val:  0.4976885612060953
X:  7
Y:  2
Val:  0.6729799629213458
X:  7
Y:  2
Val:  0.14501769904275516
X:  7
Y:  2
Val:  0.8235898364460889
X:  7
Y:  2
Val:  0.34773467038533334
X:  7
Y:  3
Val:  0.00035630464521252555
X:  7
Y:  3
Val:  nan
X:  7
Y:  3
Val:  0.5627396854324911
X:  7
Y:  3
Val:  0.49675883599122506
X:  7
Y:  3
Val:  0.6743069844713688
X:  7
Y:  3
Val:  0.14403764231380625
X:  7
Y:  3
Val:  0.8209010470131212
X:  7
Y:  3
Val:  0.3473678174097302
X:  7
Y:  4
Val:  0.00034877857465145457
X:  7
Y:  4
Val:  nan
X:  7
Y:  4
Val:  0.5646289648572647
X:  7
Y:  4
Val:  0.49582911077635505
X:  7
Y:  4
Val:  0.6756340060213917
X:  7
Y:  4
Val:  0.14305758558485734
X:  7
Y:  4
Val:  0.8182122575801537
X:  7
Y:  4
Val:  0.34700096443412715
X:  7
Y:  5
Val:  0.00034125250409038364
X:  7
Y:  5
Val:  nan
X:  7
Y:  5
Val:  0.5665182442820383
X:  7
Y:  5
Val:  0.4948993855614849
X:  7
Y:  5
Val:  0.6769610275714149
X:  7
Y:  5
Val:  0.14207752885590846
X:  7
Y:  5
Val:  0.815523468147

...

X:  1
Y:  1
Val:  0.45778164116659664
X:  1
Y:  1
Val:  0.46614585561296434
X:  1
Y:  1
Val:  0.754592584121034
X:  1
Y:  1
Val:  0.14311270420034383
X:  1
Y:  1
Val:  0.6309666883525755
X:  1
Y:  1
Val:  0.21028354699890797
X:  1
Y:  2
Val:  0.0
X:  1
Y:  2
Val:  nan
X:  1
Y:  2
Val:  0.45733509423576685
X:  1
Y:  2
Val:  0.4659750677341499
X:  1
Y:  2
Val:  0.7559301122133667
X:  1
Y:  2
Val:  0.14277392685920087
X:  1
Y:  2
Val:  0.6312335613461321
X:  1
Y:  2
Val:  0.21028354699890797
X:  1
Y:  3
Val:  0.0
X:  1
Y:  3
Val:  nan
X:  1
Y:  3
Val:  0.45688854730493716
X:  1
Y:  3
Val:  0.46580427985533535
X:  1
Y:  3
Val:  0.7572676403056992
X:  1
Y:  3
Val:  0.1424351495180579
X:  1
Y:  3
Val:  0.6315004343396887
X:  1
Y:  3
Val:  0.21028354699890797
X:  1
Y:  4
Val:  0.0
X:  1
Y:  4
Val:  nan
X:  1
Y:  4
Val:  0.4564420003741075
X:  1
Y:  4
Val:  0.4656334919765209
X:  1
Y:  4
Val:  0.7586051683980317
X:  1
Y:  4
Val:  0.14209637217691495
X:  1
Y:  4
Val:  0.6317673073332453
X:  1
Y

Y:  0
Val:  0.6326312869437298
X:  4
Y:  0
Val:  0.21085286710976617
X:  4
Y:  1
Val:  0.0
X:  4
Y:  1
Val:  nan
X:  4
Y:  1
Val:  0.45613979248021286
X:  4
Y:  1
Val:  0.4653022053221524
X:  4
Y:  1
Val:  0.7601691917237509
X:  4
Y:  1
Val:  0.14416893711086784
X:  4
Y:  1
Val:  0.632992128649285
X:  4
Y:  1
Val:  0.21085286710976617
X:  4
Y:  2
Val:  0.0
X:  4
Y:  2
Val:  nan
X:  4
Y:  2
Val:  0.4557393217572609
X:  4
Y:  2
Val:  0.46509499938019544
X:  4
Y:  2
Val:  0.7614167787697322
X:  4
Y:  2
Val:  0.14382691899060424
X:  4
Y:  2
Val:  0.6333529703548406
X:  4
Y:  2
Val:  0.21085286710976617
X:  4
Y:  3
Val:  0.0
X:  4
Y:  3
Val:  nan
X:  4
Y:  3
Val:  0.4553388510343089
X:  4
Y:  3
Val:  0.4648877934382385
X:  4
Y:  3
Val:  0.7626643658157134
X:  4
Y:  3
Val:  0.1434849008703407
X:  4
Y:  3
Val:  0.6337138120603957
X:  4
Y:  3
Val:  0.21085286710976614
X:  4
Y:  4
Val:  0.0
X:  4
Y:  4
Val:  nan
X:  4
Y:  4
Val:  0.454938380311357
X:  4
Y:  4
Val:  0.46468058749628166
X:  4
Y: 

Y:  3
Val:  0.4639711753697209
X:  7
Y:  3
Val:  0.7680618665544359
X:  7
Y:  3
Val:  0.14453480301728275
X:  7
Y:  3
Val:  0.6359275077283375
X:  7
Y:  3
Val:  0.21142226900230723
X:  7
Y:  4
Val:  0.0
X:  7
Y:  4
Val:  nan
X:  7
Y:  4
Val:  0.4534345442566247
X:  7
Y:  4
Val:  0.4637275461332404
X:  7
Y:  4
Val:  0.7692194996342162
X:  7
Y:  4
Val:  0.1441895436523672
X:  7
Y:  4
Val:  0.636382331644307
X:  7
Y:  4
Val:  0.21142226900230723
X:  7
Y:  5
Val:  0.0
X:  7
Y:  5
Val:  nan
X:  7
Y:  5
Val:  0.4530801563603047
X:  7
Y:  5
Val:  0.46348391689676
X:  7
Y:  5
Val:  0.7703771327139967
X:  7
Y:  5
Val:  0.1438442842874516
X:  7
Y:  5
Val:  0.6368371555602765
X:  7
Y:  5
Val:  0.21142226900230726
X:  7
Y:  6
Val:  0.0
X:  7
Y:  6
Val:  nan
X:  7
Y:  6
Val:  0.45272576846398466
X:  7
Y:  6
Val:  0.4632402876602796
X:  7
Y:  6
Val:  0.7715347657937771
X:  7
Y:  6
Val:  0.14349902492253602
X:  7
Y:  6
Val:  0.6372919794762459
X:  7
Y:  6
Val:  0.21142226900230723
X:  7
Y:  7
Val:  0

Y:  10
Val:  0.14280287837526978
X:  9
Y:  10
Val:  0.6410258850087647
X:  9
Y:  10
Val:  0.21180191569368698
X:  9
Y:  11
Val:  0.0
X:  9
Y:  11
Val:  nan
X:  9
Y:  11
Val:  0.45016623055707244
X:  9
Y:  11
Val:  0.46121670881420096
X:  9
Y:  11
Val:  0.7804418837555049
X:  9
Y:  11
Val:  0.14245545792198375
X:  9
Y:  11
Val:  0.6415433712300405
X:  9
Y:  11
Val:  0.21180191569368703
X:  9
Y:  12
Val:  0.0
X:  9
Y:  12
Val:  nan
X:  9
Y:  12
Val:  0.4498425682218885
X:  9
Y:  12
Val:  0.46094879447534176
X:  9
Y:  12
Val:  0.7815395403475146
X:  9
Y:  12
Val:  0.14210803746869768
X:  9
Y:  12
Val:  0.642060857451316
X:  9
Y:  12
Val:  0.211801915693687
X:  10
Y:  0
Val:  0.0
X:  10
Y:  0
Val:  nan
X:  10
Y:  0
Val:  0.45316368604115376
X:  10
Y:  0
Val:  0.46389459895009294
X:  10
Y:  0
Val:  0.7702571438483307
X:  10
Y:  0
Val:  0.14663035916691528
X:  10
Y:  0
Val:  0.6364950624376398
X:  10
Y:  0
Val:  0.21199175266725317
X:  10
Y:  1
Val:  0.0
X:  10
Y:  1
Val:  nan
X:  10
Y:  1
V

Val:  0.45147503382844284
X:  13
Y:  0
Val:  0.46308701889006776
X:  13
Y:  0
Val:  0.7759261341850535
X:  13
Y:  0
Val:  0.147690289378508
X:  13
Y:  0
Val:  0.6384273662838588
X:  13
Y:  0
Val:  0.21256131809531809
X:  13
Y:  1
Val:  0.0
X:  13
Y:  1
Val:  nan
X:  13
Y:  1
Val:  0.4512128314386448
X:  13
Y:  1
Val:  0.46277052737277136
X:  13
Y:  1
Val:  0.7769038205787482
X:  13
Y:  1
Val:  0.14733854612790562
X:  13
Y:  1
Val:  0.6390701951097764
X:  13
Y:  1
Val:  0.21256131809531809
X:  13
Y:  2
Val:  0.0
X:  13
Y:  2
Val:  nan
X:  13
Y:  2
Val:  0.45095062904884664
X:  13
Y:  2
Val:  0.46245403585547484
X:  13
Y:  2
Val:  0.7778815069724426
X:  13
Y:  2
Val:  0.14698680287730323
X:  13
Y:  2
Val:  0.6397130239356938
X:  13
Y:  2
Val:  0.21256131809531809
X:  13
Y:  3
Val:  0.0
X:  13
Y:  3
Val:  nan
X:  13
Y:  3
Val:  0.45068842665904874
X:  13
Y:  3
Val:  0.4621375443381783
X:  13
Y:  3
Val:  0.7788591933661374
X:  13
Y:  3
Val:  0.14663505962670084
X:  13
Y:  3
Val:  0.6403558

In [213]:
fireline

matrix([[  0,   0,   0,   0,   0],
        [  0, 154, 154, 154,   0],
        [  0, 154, 154,   0,   0],
        [  0,   0, 155, 155,   0],
        [  0, 155, 155, 155, 155],
        [  0,   0,   0, 156, 156]])

In [218]:
fireline[2,3]

0